In [16]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [17]:
from crewai import Agent, Task, Crew


In [18]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()


In [19]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


In [20]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [21]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [22]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [23]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [24]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)


In [25]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [26]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [27]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)


In [28]:
from crewai import Crew, Process # Process class let's you define different ways for agents to work together
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)


2024-10-31 10:08:47,543 - 8382724160 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [29]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'NU',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}


In [30]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (NU). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is continuously monitoring and analyzing market data for NU to identify trends and predict market movements. I should delegate specific tasks to the team members based on their expertise and ask relevant questions to gather insights.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for NU using statistical modeling and machine learning", "context": "Provide insights and alerts about significant market opportunities or threats for NU."}

> Entering new CrewAgentExecutor chain...
Thought: First, I need to gather real-time market data for the stock ticker "NU" to monitor and analyze its performance for any

In [31]:
from IPython.display import Markdown
Markdown(result)


Risk Analysis Report for NU Trading Strategies and Execution Plans:

Introduction:
The following report provides a comprehensive analysis of the potential risks associated with the proposed trading strategies and execution plans for NU. It also includes detailed mitigation recommendations to manage and minimize these risks effectively. The analysis is based on the detailed execution plans provided for Momentum Trading, Moving Average Crossover, Scalping, VWAP Strategy, and Risk Management.

Risk Analysis:

1. Transmission and Cut-Off Times Risk:
- Risk: Orders placed after the 13.00 cut-off time may not be executed the same day, leading to unfavorable price movements.
- Mitigation: Ensure timely placement of trades and implement automated reminders for traders to manage deadlines effectively.

2. Best Execution Factors Risk:
- Risk: Complexity and interplay of best execution factors could lead to suboptimal execution.
- Mitigation: Conduct continuous training for traders on optimizing factors and regular review of execution quality.

3. Non-UK Shares Currency Risk:
- Risk: Currency fluctuations can impact the GBP value of trades for non-UK shares.
- Mitigation: Implement hedging strategies and monitor currency exposures regularly.

4. Order Aggregation and Priority Risk:
- Risk: Client orders may be aggregated, occasionally working to a client's disadvantage.
- Mitigation: Clearly communicate aggregation policy impacts to clients and ensure fairness in order allocation.

5. Dilution Levy and Exit Charges Risk:
- Risk: Dilution levy and unexpected exit charges could increase costs.
- Mitigation: Provide pre-trade transparency on potential charges and manage trade sizes actively.

6. Foreign Exchange Execution Risk:
- Risk: Non-UK shares involve foreign exchange transactions that may not be executed at favorable rates.
- Mitigation: Utilize pre-agreed forex contracts or seek better forex execution terms.

7. Regulatory and Compliance Risk:
- Risk: The execution policy must align with changing regulatory requirements.
- Mitigation: Conduct regular updates and compliance checks to ensure policy compliance with current regulations.

Conclusion:
The detailed risk analysis and mitigation recommendations aim to assist in managing and minimizing the risks associated with NU's trading strategies and execution plans. It is crucial to regularly review and adapt these strategies to align with the dynamic market conditions and regulatory changes for optimal risk management and trading performance.